# MOFA+: training a model in Python
Author: Ricard Argelaguet.    
Affiliation: European Bioinformatics Institute, Cambridge, UK.  
Date: 13/11/2019  



This notebook contains a detailed tutorial on how to train MOFA using Python.
A template script to run the code below can be found [here](https://github.com/bioFAM/MOFA2/blob/master/template_script.py)

## 1) Load libraries

In [12]:
from mofapy2.run.entry_point import entry_point
import pandas as pd
import numpy as np

# initialise the entry point
ent = entry_point()


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        


## 2) Load data

To create a MOFA+ object you need to specify four dimensions: samples (cells), features, view(s) and group(s). MOFA objects can be created from a wide range of input formats:

### 2.1) pandas data.frame format
A pandas data.frame with columns `sample`, `group`, `feature`, `view`, `value`.  This is the most intuitive format, as it summarises all omics/groups in a single data structure. Also, there is no need to add rows that correspond to missing data.

For example:
```
sample  group   feature value   view
sample1  groupA    gene1   2.8044  RNA
sample1  groupA    gene3    2.2069  RNA
sample2  groupB    gene2    0.1454  RNA
sample2  groupB    gene1     2.7021  RNA
sample2  groupB    promoter1    3.8618  Methylation
sample3  groupB    promoter2    3.2545  Methylation
sample3  groupB    promoter3   1.5014  Methylation
```

Here we load a simulated data set with the following dimensions:

In [13]:
D = [1000,1000] # Number of features per view
M = len(D)      # Number of views
K = 5           # Number of factors
N = [100,100]   # Number of samples per group
G = len(N)      # Number of groups

data_dt = pd.read_csv("http://ftp.ebi.ac.uk/pub/databases/mofa/getting_started/data.txt.gz", sep="\t")
data_dt.head()

,sample,group,feature,view,value
0,sample_0_group_0,group_0,feature_0_view_0,view_0,-2.05
1,sample_1_group_0,group_0,feature_0_view_0,view_0,0.10
2,sample_2_group_0,group_0,feature_0_view_0,view_0,1.44
3,sample_3_group_0,group_0,feature_0_view_0,view_0,-0.28
4,sample_4_group_0,group_0,feature_0_view_0,view_0,-0.88


### 2.2) List of matrices
A nested list of numpy arrays, where the first index refers to the view and the second index refers to the group. Samples are stored in the rows and features are stored in the columns. All views for a given group G must have the same samples in the rows. If there is any sample that is missing a particular view, the column needs to be filled with NAs.

Loading the same data above in matrix format:

In [14]:
data_prefix = "http://ftp.ebi.ac.uk/pub/databases/mofa/getting_started"
data_mat = [[None for g in range(G)] for m in range(M)]
for m in range(M):
    for g in range(G):
        data_mat[m][g] = np.loadtxt("%s/%d_%d.txt.gz" % (data_prefix,m,g), delimiter="\t")

### 2.3 Define data options
- **scale_groups**: if groups have different ranges/variances, it is good practice to scale each group to unit variance. Default is False
- **scale_views**: if views have different ranges/variances, it is good practice to scale each view to unit variance. Default is False

In [15]:
ent.set_data_options(
    scale_groups = False, 
    scale_views = False
)

### 2.4) Add the data to the model

This has to be run after defining the data options
- **likelihods**: a list of strings, either "gaussian", "poisson" or "bernoulli". If None (default), they are guessed internally

In [16]:
# option 1: data.frame format (slower)
ent.set_data_df(data_dt, likelihoods = ["gaussian","gaussian"])

# option 2: nested matrix format (faster)
ent.set_data_matrix(data_mat, likelihoods = ["gaussian","gaussian"])

# AnnData format
# (...)

View names not provided, using default naming convention:
- view1, view2, ..., viewM

Features names not provided, using default naming convention:
- feature1_view1, featureD_viewM

Groups names not provided, using default naming convention:
- group1, group2, ..., groupG

Samples names not provided, using default naming convention:
- sample1_group1, sample2_group1, sample1_group2, ..., sampleN_groupG

Successfully loaded view='view0' group='group0' with N=100 samples and D=1000 features...
Successfully loaded view='view0' group='group1' with N=100 samples and D=1000 features...
Successfully loaded view='view1' group='group0' with N=100 samples and D=1000 features...
Successfully loaded view='view1' group='group1' with N=100 samples and D=1000 features...




## 3) Set model options

- **factors**: number of factors
- **spikeslab_weights**: use spike-slab sparsity prior in the weights? default is TRUE
- **ard_factors**: use ARD prior in the factors? Default is TRUE if using multiple groups. This is ugessed by default.
- **ard_weights**: use ARD prior in the weights? Default is TRUE if using multiple views. This is guessed by default

Only change the default model options if you are familiar with the underlying mathematical model!


In [17]:
ent.set_model_options(
    factors = 10, 
    spikeslab_weights = True, 
    ard_factors = True,
    ard_weights = True
)

Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the weights: True
- Spike-and-slab prior on the factors: False
- Spike-and-slab prior on the weights: True 

Likelihoods:
- View 0 (view0): gaussian
- View 1 (view1): gaussian


## 4) Set training options

- **iter**: number of iterations. Default is 1000.
- **convergence_mode**: "fast", "medium", "slow". For exploration, the fast mode is good enough.
- **startELBO**: initial iteration to compute the ELBO (the objective function used to assess convergence)
- **freqELBO**: frequency of computations of the ELBO (the objective function used to assess convergence)
- **dropR2**: minimum variance explained criteria to drop factors while training
gpu_mode: use GPU mode? (needs cupy installed and a functional GPU, see https://cupy.chainer.org/)
- **verbose**: verbose mode?
- **seed**: random seed

In [18]:
ent.set_train_options(
    iter = 1000, 
    convergence_mode = "fast", 
    startELBO = 1, 
    freqELBO = 1, 
    dropR2 = 0.001, 
    gpu_mode = True, 
    verbose = False, 
    seed = 1
)


GPU mode is activated, but GPU not found... switching to CPU mode
For GPU mode, you need:
1 - Make sure that you are running MOFA+ on a machine with an NVIDIA GPU
2 - Install CUPY following instructions on https://docs-cupy.chainer.org/en/stable/install.html



##### 5) (Optional)  stochastic inference options

If the number of samples is very large (at the order of >1e4), you may want to try the stochastic inference scheme. However, it requires some additional hyperparameters that in some data sets may need to be optimised (see the [stochastic vignette](https://raw.githack.com/bioFAM/MOFA2/master/MOFA2/vignettes/stochastic_inference.html)).

- **batch_size**: float value indicating the batch size (as a fraction of the total data set: either 0.10, 0.25 or 0.50)
- **learning_rate**: learning rate (we recommend values from 0.25 to 0.50)
- **forgetting_rate**: forgetting rate (we recommend values from 0.75 to 1.0)


In [22]:
# We do not want to use stochastic inference for this data set

# ent.set_stochastic_options(
# batch_size = 0.5,
# learning_rate = 0.5, 
# forgetting_rate = 0.25
#)

## 6) Build and train the MOFA object 

After training, the model will be saved as an hdf5 file

In [19]:
ent.build()

ent.run()

# Save the output
outfile = "/Users/ricard/data/mofaplus/hdf5/test.hdf5"
ent.save(outfile)



######################################
## Training the model with seed 1 ##
######################################


ELBO before training: -5381458.89 

Iteration 1: time=0.06, ELBO=-575232.56, deltaELBO=4806226.336 (89.31084362%), Factors=9
Iteration 2: time=0.07, ELBO=-475536.26, deltaELBO=99696.298 (1.85258867%), Factors=8
Iteration 3: time=0.06, ELBO=-467044.81, deltaELBO=8491.451 (0.15779088%), Factors=7
Iteration 4: time=0.05, ELBO=-462583.09, deltaELBO=4461.713 (0.08290899%), Factors=6
Iteration 5: time=0.06, ELBO=-459116.18, deltaELBO=3466.915 (0.06442334%), Factors=5
Iteration 6: time=0.04, ELBO=-458285.22, deltaELBO=830.956 (0.01544108%), Factors=5
Iteration 7: time=0.04, ELBO=-457814.07, deltaELBO=471.158 (0.00875520%), Factors=5
Iteration 8: time=0.04, ELBO=-457487.58, deltaELBO=326.485 (0.00606685%), Factors=5
Iteration 9: time=0.06, ELBO=-457202.99, deltaELBO=284.587 (0.00528829%), Factors=5
Iteration 10: time=0.04, ELBO=-456926.93, deltaELBO=276.062 (0.00512988%), Fact

## 7) Downstream analysis

This finishes the tutorial on how to train a MOFA object from python. To continue with the downstream analysis you have to switch to R. Please, follow [this tutorial](https://raw.githack.com/bioFAM/MOFA2/master/MOFA2/vignettes/downstream_analysis.html)